# 1. 라이브러리 호출

In [1]:
import pandas as pd
import re
import kss
from konlpy.tag import Kkma
from konlpy.tag import Okt
from tqdm import tqdm
import time

kkma = Kkma()
okt = Okt()
#spacing = Spacing()

# 2. 관련 함수 정의

## 2-(1) 형태소 분석

In [2]:
#감성분석 전 정제 함수(형태소 분석)
def refining(reviews):
    result_list = []
    for r in reviews:
        review = re.sub(' ', 's', r)
        review = okt.pos(review, stem=True, norm=True) #형태소 분석
        string = ''
        for word in review:
            string += word[0]
        string = re.sub('s', ' ', string)
        result_list.append(string)
    return result_list

## 2-(2) 감성 사전 정의

In [3]:
#감성사전
service_good_feature = {'서비스':['좋','친절','괜찮','최고','빠르','짱','훌륭','추천','감사','구수','최상','대박',
                               '훈훈','특별','개이득','최고','만족','세련','최고','감동'],
                        '사장':['친절','스윗','센스'],
                        '알바':['친절','스윗','센스'],
                        '직원': ['친절','스윗','센스'],
                        '다들': ['친절','스윗','센스'],
                        '일을':['잘','빠르게'],
                        '일도':['잘','빠르게'],
                        '서빙':['잘','빠르게']}

service_bad_feature = {'서비스': ['아쉽','최악','나쁘','느리','빡치','비추','별로','그냥','낙제','쏘다쏘다','엉망','실망','불친절','문제','컴플레인',
                               '거지','그닥','그다지','구려','불편','엉성','헬','개판'],
                       '알바':['불친절','똑바로','재수'],
                       '사장':['불친절','똑바로','재수'],
                       '직원':['불친절','똑바로','재수'],
                       '다들': ['불친절','똑바로','재수'],
                      '일을':['못','느리게','답답'],
                      '일도':['못','느리게', '천천히'],
                      '서빙':['못','느리게','천천히','답답']}

taste_good_feature = {'간':['맞','적절','딱','환상','담백'],
                      '음식':['깔끔'],
                      '맛':['있','좋','나다','최고']}

taste_bad_feature = {'간':['세','쎄','강하다','별로'],
                     '음식':['별로','쏘다쏘다','최악'],
                     '맛':['별로','최악']}

taste_good_emotion = ['고소','부드럽','신선','촉촉','싱싱','정갈','최고']

taste_bad_emotion = ['싱겁','느끼다하다','짜다','느끼다','짜다','딱딱하다','차갑다']

cost_good_feature = {'가격': ['괜찮','착하다','저렴','적당','싸다','좋다','합리적','훌륭','최고','만족','마음','든든','알맞다',
                            '무난','괜춘','최상','최상','굿','엄지','낮'],
                     '가성비':['괜찮','착하다','저렴','적당','싸다','좋다','합리적','훌륭','최고','만족','마음','든든','알맞다',
                            '무난','괜춘','최상','최상','굿','엄지'],
                     '양':['많','적당','푸짐하고','괜찮다','넉넉','충분','든든']
                     }

cost_bad_feature ={'가격': ['비싸','있다','나쁘','사악','비효율','높다','부담','아쉽','쏘다쏘다','별로','그닥','그다지','쎄','ㅎㄷㄷ','높','거품'],
                   '가성비':['별로'],
                   '양':['적다','작다','아쉽','적다','다소','별로'],
                   }

atmosphere_good_feature = {'분위기': ['좋','괜찮','조용','깔끔','적당','깡패','굉장','아담','완벽','아기자기','고급','최고','세련','만족','아늑','훌륭','예쁘','이쁘','짱',
                                   '심쿵','따뜻','깨끗','독특','매력','모던','취향저격','맘','마음','클래식','아름','인상적','귀엽','포근'], 
                           '인테리어': ['좋','괜찮','조용','깔끔','적당','깡패','굉장','아담','완벽','아기자기','고급','최고','세련','만족','아늑','훌륭','예쁘','이쁘','짱',
                                    '심쿵','따뜻','깨끗','독특','매력','모던','취향저격','맘','마음','클래식','아름','인상적','귀엽','포근'],
                          '매장': ['좋','괜찮','조용','깔끔','적당','깡패','굉장','아담','완벽','아기자기','고급','최고','세련','만족','아늑','훌륭','예쁘','이쁘','짱',
                                   '심쿵','따뜻','깨끗','독특','매력','모던','취향저격','맘','마음','클래식','아름','인상적','귀엽','포근', '넓']}

atmosphere_bad_feature = {'분위기': ['나쁘다','바쁘다','어수선하다','이상하다','촌스럽다','별로','부담스럽다','시끄럽','복잡' ],
                          '인테리어':[],
                         '매장':[]}

visit_good_feature = {'의사':['있다','충만','백프로','백프롭','많','만땅','마구','그득','만점','넘침'],
                      '다시':['가다','오다','방문','찾다','가보다','한번','갈다','찾아가다','가야지','갈거다','방문하다보고',
                            '생각나다','방문한다면','와보고','재방문','접하다','간다면','갈다때가','먹다고프다','방문한다임','오자고','가기로','갈다생각이다','가면'],
                      '굳이':[]}

visit_bad_feature = {'의사':['글쎄'],
                     '굳이':['다시','많이','여기까지','줄서서','찾아','시키다','가다','찾다','여기','기다리다','줄을','사먹'],
                     '다시':[]}

negative_word_emotion = ['안 ','않','못','없','아닌','아니']

## 2-(3) 각 특성별(서비스, 맛, 가격, 분위기, 재방문 의사) 키워드 추출

In [4]:
#각 특성별 키워드 추출, 문장 속에 다른 단어들과 섞여서 쓰이는 형태를 보고 감성분석을 진행해야 하니까 키워드+주변어절 추출합니다
#구 단위분석
def get_feature_keywords(feature_keywords, review):
    feature_temp = []
    for keyword in feature_keywords:
        if re.findall(keyword, review): # 리뷰에서 키워드 있는거 다 찾아서
            sub_list = ['게','고','음','며','데','만','도','면'] #연결어미
            
            for sub in sub_list:
                if sub+' ' in review: # 연결어미가 띄어쓰기랑 같이 있으면
                    review = re.sub(sub+' ', sub+',', review) # 연결어미 + 띄어쓰기를 연결어미 + , 로 대체
                    #키워드와 의미 없는 연결어미가 함께 추출되는 일이 없도록(밑에서 어절 단위로 추출함)
                
            a = re.findall(keyword +'+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review) # K한 한 한글
            # 키워드+조사 띄어쓰기 어절 2개가 붙어 있는거 다 찾기
            b = re.findall(keyword + '+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review) # K 한 한글 
            # 키워드와 띄어쓰기 어절 1개짜리 다 찾기
            c = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+' + keyword +'[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review) # 한 K한글 예쁜 분위기가
            # 어절 + 키워드 + 어절인 부분 다 찾기
            
            # 추출한 키워드들을 feature_temp에 append
            for ngram in a:
                t = ()
                feature_temp.append(t + (ngram,keyword))
            for ngram in b:
                t = ()
                feature_temp.append(t + (ngram,keyword))
            for ngram in c:
                t = ()
                feature_temp.append(t + (ngram,keyword))
                
    return feature_temp

## 2-(4) 각 특성에 대한 긍정 및 부정 감정 추출

In [5]:
#특성별 감성분석 인풋으로 특성별 긍정사전, 부정사전, 추출한 키워드
def get_feature_emotions(feature_good_dict,feature_bad_dict, feature_temp):
    good_feature_emotion_list = []
    bad_feature_emotion_list = []
    
    for ngrams in feature_temp: # 추출한 키워드(추출 문장, 키워드)에서
        keyword = ngrams[1] # 키워드 -> 감성사전에서 특성별 긍부정을 추출할 키워드
        ngram = ngrams[0] # 추출 문장 -> 특성별 긍부정 단어가 있는지 확인할 문장
        is_bad_feature = None
        
        good_emotion_list = feature_good_dict[keyword] # 특성별 긍정 감정어 
        bad_emotion_list = feature_bad_dict[keyword] # 특성별 부정 감정어
        for emotion in good_emotion_list: # 긍정감정어에서
            if re.findall(emotion, ngram): # 긍정어와 추출한 문장이 함께 있다면
                is_bad_feature = False  # 나쁜 특성은 아니다
        for emotion in bad_emotion_list: # 부정감정어에서
            if re.findall(emotion, ngram): # 부정어와 추출한 문장이 함께 있다면
                is_bad_feature = True    # 이 문장은 부정문이다
        for negative in negative_word_emotion: #[안, 않, 못, 없 등]의 부정어에서
            if re.findall(negative, ngram): # 부정어와 추출한 문장이 함께 있으면서
                if is_bad_feature == True: # 이 문장이 부정문이면
                    is_bad_feature = False # 이 문장은 사실 부정한 문장이 아니다
                    break
                elif is_bad_feature == False: # 이 문장이 부정한 문장이 아니면
                    is_bad_feature = True # 이 문장은 사실 부정한 문장이다
                    break
                else:
                    is_bad_feature = True # 그냥 아무 감정 없는 평범한 문장이라면 부정한 문장이다
                    break   
        if is_bad_feature:
            bad_feature_emotion_list.append(ngram) # 부정 문장에 추가
        elif is_bad_feature == False:
            good_feature_emotion_list.append(ngram) # 긍정 문장에 추가
        else:
            pass # 아무것도 아니면 패스
    return good_feature_emotion_list, bad_feature_emotion_list

## 2-(5) 맛 특성에 대한 긍정 및 부정 감정 추출

In [6]:
#맛 긍부정 분석 맛사전의 경우 feature 사전과 emotions 사전이 나눠져 있어서 두번 진행
#맛 emotions 사전에 대한 처리
def get_taste_emotion(taste_good_emotions, taste_bad_emotions):
    bad_taste_emotion_list = []
    good_taste_emotion_list = []
    for ngrams in taste_good_emotions:
        ngram = ngrams[0]
        is_bad_taste = False
        for negative in negative_word_emotion:
            if re.findall(negative,ngram):
                is_bad_taste = True
        if is_bad_taste:
            bad_taste_emotion_list.append(ngram)
        else:
            good_taste_emotion_list.append(ngram)
    
    for ngrams in taste_bad_emotions:
        ngram = ngrams[0]
        is_bad_taste = True
        for negative in negative_word_emotion:
            if re.findall(negative,ngram):
                is_bad_taste = False
        if is_bad_taste:
            bad_taste_emotion_list.append(ngram)
        else:
            good_taste_emotion_list.append(ngram)
            
    return good_taste_emotion_list, bad_taste_emotion_list

# 3. 블로그 리뷰 감성 분석

## 3-(1) 블로그 리뷰 불러오기

In [7]:
df1 = pd.read_csv('processed_place_blog_review3.csv', encoding='utf-8-sig')

In [8]:
for i in range(len(df1['blog review'])):
            df1['blog review'][i] = str(df1['blog review'][i])

c:\Users\jhu56\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## 3-(2) 식당별로 리뷰 모으기

In [9]:
df2 = df1.groupby('name', sort=False).agg({'blog review': ','.join}).reset_index()

In [10]:
review = df2['blog review']
name = df2['name']

In [11]:
#식당 별로로 리뷰 모으기
restaurant_review = []
for i in review:
    restaurant_review.append(i.split(','))

In [12]:
for review in df1['blog review']:
    service_temp = get_feature_keywords(service_good_feature.keys(), review)
    good_service,bad_service = get_feature_emotions(service_good_feature, service_bad_feature, service_temp)
    print("=============service temp=============")
    print(service_temp)
    print("=============good service=============")
    print(good_service)
    print("=============bad service=============")
    print(bad_service)

=============service temp=============
[('사장 과 매니저', '사장')]
=============good service=============
[]
=============bad service=============
[]
=============service temp=============
[]
=============good service=============
[]
=============bad service=============
[]
=============service temp=============
[('사장 과 매니저', '사장')]
=============good service=============
[]
=============bad service=============
[]
=============service temp=============
[('사장 과 매니저', '사장')]
=============good service=============
[]
=============bad service=============
[]
=============service temp=============
[]
=============good service=============
[]
=============bad service=============
[]
=============service temp=============
[]
=============good service=============
[]
=============bad service=============
[]
=============service temp=============
[('사장 컨셉 꾸며', '사장')]
=============good service=============
[]
=============bad service=============
[]
=============service temp=============
[('사장 다 인테리어',

## 3-(3) 블로그 리뷰 감성 분석 결과 

In [13]:
#식당별 감성분석 결과 카운팅 및 스코어
#check_division은 restaurant_good_service_count, restaurant_good_service_count + restaurant_bad_service_count 파라미터로 받고, *100 함 -> 점수가 나옴
check_division = lambda x, y: y if y ==0 else round((x / float(y)),2) # x,y 파라미터를 받아서 y가 0이면 0, 아니면 x/y의 소수점 밑 두자리 수까지 반올림

for i in range(len(name)):
    restaurant_good_service_count = 0
    restaurant_bad_service_count = 0
    restaurant_good_atmosphere_count =0
    restaurant_bad_atmosphere_count =0
    restaurant_good_cost_count =0
    restaurant_bad_cost_count =0
    restaurant_good_visit_count = 0
    restaurant_bad_visit_count = 0
    restaurant_good_taste_count = 0
    restaurant_bad_taste_count = 0
    print(name[i]) # 식당 이름부터 출력
    reviews_list = refining(restaurant_review[i]) #식당에 대한 리뷰 전체 형태소 분석
    for review in reviews_list: #한번 돌 때마다 한 식당의 전체 리뷰에 대한 감성분석 진행
        service_temp = get_feature_keywords(service_good_feature.keys(), review)
        good_service,bad_service = get_feature_emotions(service_good_feature, service_bad_feature, service_temp)

        atmosphere_temp = get_feature_keywords(atmosphere_good_feature.keys(), review)
        good_atmosphere,bad_atmosphere = get_feature_emotions(atmosphere_good_feature, atmosphere_bad_feature, atmosphere_temp)

        cost_temp = get_feature_keywords(cost_good_feature.keys(), review)
        good_cost,bad_cost = get_feature_emotions(cost_good_feature, cost_bad_feature, cost_temp)

        visit_temp = get_feature_keywords(visit_good_feature.keys(), review)
        good_visit,bad_visit = get_feature_emotions(visit_good_feature, visit_bad_feature, visit_temp)

        taste_temp = get_feature_keywords(taste_good_feature.keys(), review)
        good_taste,bad_taste = get_feature_emotions(taste_good_feature, taste_bad_feature, taste_temp) # 맛 특성 긍부정 분석
        taste_good_emotion_temp = get_feature_keywords(taste_good_emotion, review) # 맛에 대한 긍정 감정 키워드 추출
        taste_bad_emotion_temp = get_feature_keywords(taste_bad_emotion, review) # 맛에 대한 부정 감정 키워드 추출
        good_taste2, bad_taste2 = get_taste_emotion(taste_good_emotion_temp,taste_bad_emotion_temp)
        good_taste.extend(good_taste2) # 긍정 맛 문장에 추가
        bad_taste.extend(bad_taste2) # 부정 맛 문장에 추가
        
        # 각 특성 긍부정 리스트의 크기에 따라 점수 카운트(부정은 출력 안하긴함)
        if len(good_service) > len(bad_service):
            restaurant_good_service_count += 1
        elif len(good_service) < len(bad_service):
            restaurant_bad_service_count += 1
        else:
            pass
        
        if len(good_atmosphere) > len(bad_atmosphere):
            restaurant_good_atmosphere_count += 1
        elif len(good_atmosphere) < len(bad_atmosphere):
            restaurant_bad_atmosphere_count += 1
        else:
            pass
        
        if len(good_cost) > len(bad_cost):
            restaurant_good_cost_count += 1
        elif len(good_cost) < len(bad_cost):
            restaurant_bad_cost_count += 1
        else:
            pass
            
        if len(good_visit) > len(bad_visit):
            restaurant_good_visit_count += 1
        elif len(good_visit) < len(bad_visit):
            restaurant_bad_visit_count += 1
        else:
            pass
        
        if len(good_taste) > len(bad_taste):
            restaurant_good_taste_count += 1
        elif len(good_taste) < len(bad_taste):
            restaurant_bad_taste_count += 1
        else:
            pass
        
    TT = restaurant_good_service_count + restaurant_bad_service_count + restaurant_good_taste_count + restaurant_bad_taste_count + restaurant_good_atmosphere_count + restaurant_bad_atmosphere_count + restaurant_good_cost_count + restaurant_bad_cost_count
    
    #if TT > 5: # 총 감성 카운트가 5 이상만 출력하는건데 일단 주석 처리 해 놓고 전체 출력
    print('Total review count: {}'.format(len(restaurant_review[i])))
    print('Good service: {}/{} = {}'.format(restaurant_good_service_count,restaurant_good_service_count + restaurant_bad_service_count,100*check_division(restaurant_good_service_count, restaurant_good_service_count + restaurant_bad_service_count)))
    print('Good atmosphere: {}/{} = {}'.format(restaurant_good_atmosphere_count,restaurant_good_atmosphere_count + restaurant_bad_atmosphere_count,100*check_division(restaurant_good_atmosphere_count,restaurant_good_atmosphere_count + restaurant_bad_atmosphere_count))) 
    print('Good cost: {}/{} = {}'.format(restaurant_good_cost_count,restaurant_good_cost_count + restaurant_bad_cost_count, 100*check_division(restaurant_good_cost_count,restaurant_good_cost_count + restaurant_bad_cost_count)))
    print('Good taste: {}/{} = {}'.format(restaurant_good_taste_count,restaurant_good_taste_count + restaurant_bad_taste_count,100*check_division(restaurant_good_taste_count,restaurant_good_taste_count + restaurant_bad_taste_count)))
    print('Good revisit: {}/{} = {}'.format(restaurant_good_visit_count,restaurant_good_visit_count+restaurant_bad_visit_count,100*check_division(restaurant_good_visit_count,restaurant_good_visit_count+restaurant_bad_visit_count)))
    print('')

배곧 또잇카페 배곧점
Total review count: 44
Good service: 3/4 = 75.0
Good atmosphere: 23/23 = 100.0
Good cost: 1/1 = 100.0
Good taste: 31/34 = 91.0
Good revisit: 1/2 = 50.0

배곧 카츠오모이
Total review count: 84
Good service: 8/9 = 89.0
Good atmosphere: 1/1 = 100.0
Good cost: 2/3 = 67.0
Good taste: 62/69 = 90.0
Good revisit: 1/2 = 50.0

배곧 양푼애등갈비
Total review count: 70
Good service: 5/5 = 100.0
Good atmosphere: 3/9 = 33.0
Good cost: 8/19 = 42.0
Good taste: 63/65 = 97.0
Good revisit: 0/2 = 0.0

배곧 투파인드피터 배곧점
Total review count: 85
Good service: 1/1 = 100.0
Good atmosphere: 32/34 = 94.0
Good cost: 7/13 = 54.0
Good taste: 60/69 = 87.0
Good revisit: 1/4 = 25.0

배곧 소쇄원 배곧본점
Total review count: 30
Good service: 1/1 = 100.0
Good atmosphere: 3/3 = 100.0
Good cost: 6/7 = 86.0
Good taste: 27/28 = 96.0
Good revisit: 0/0 = 0

배곧 정동진
Total review count: 81
Good service: 1/2 = 50.0
Good atmosphere: 2/2 = 100.0
Good cost: 10/11 = 91.0
Good taste: 55/65 = 85.0
Good revisit: 1/1 = 100.0

배곧 Tailor.Button
Total review

# 4. 방문자 리뷰 감성 분석

## 4-(1) 방문자 리뷰 불러오기

In [14]:
df3 = pd.read_csv('processed_place_visitor_review4.csv', encoding='utf-8-sig')

## 4-(2) 식당 별로 리뷰 모으기

In [15]:
df4 = df3.groupby('name', sort=False).agg({'visitor review': ','.join}).reset_index()

In [16]:
review = df4['visitor review']
name = df4['name']

In [17]:
restaurant_review = []
for i in review:
    restaurant_review.append(i.split(','))

## 4-(3) 네이버 플레이스 방문자 리뷰 감성 분석 결과

In [18]:
#식당별 감성분석 결과 카운팅 및 스코어
#check_division은 restaurant_good_service_count, restaurant_good_service_count + restaurant_bad_service_count 파라미터로 받고, *100 함 -> 점수가 나옴
check_division = lambda x, y: y if y ==0 else round((x / float(y)),2) # x,y 파라미터를 받아서 y가 0이면 0, 아니면 x/y의 소수점 밑 두자리 수까지 반올림

for i in range(len(name)):
    restaurant_good_service_count = 0
    restaurant_bad_service_count = 0
    restaurant_good_atmosphere_count =0
    restaurant_bad_atmosphere_count =0
    restaurant_good_cost_count =0
    restaurant_bad_cost_count =0
    restaurant_good_visit_count = 0
    restaurant_bad_visit_count = 0
    restaurant_good_taste_count = 0
    restaurant_bad_taste_count = 0
    print(name[i]) #식당 이름부터 출력
    reviews_list = refining(restaurant_review[i]) #식당에 대한 리뷰 전체 형태소 분석
    for review in reviews_list: #한번 돌 때마다 한 식당의 전체 리뷰에 대한 감성분석 진행
        service_temp = get_feature_keywords(service_good_feature.keys(), review)
        good_service,bad_service = get_feature_emotions(service_good_feature, service_bad_feature, service_temp)

        atmosphere_temp = get_feature_keywords(atmosphere_good_feature.keys(), review)
        good_atmosphere,bad_atmosphere = get_feature_emotions(atmosphere_good_feature, atmosphere_bad_feature, atmosphere_temp)

        cost_temp = get_feature_keywords(cost_good_feature.keys(), review)
        good_cost,bad_cost = get_feature_emotions(cost_good_feature, cost_bad_feature, cost_temp)

        visit_temp = get_feature_keywords(visit_good_feature.keys(), review)
        good_visit,bad_visit = get_feature_emotions(visit_good_feature, visit_bad_feature, visit_temp)

        taste_temp = get_feature_keywords(taste_good_feature.keys(), review)
        good_taste,bad_taste = get_feature_emotions(taste_good_feature, taste_bad_feature, taste_temp) # 맛 특성 긍부정 분석
        taste_good_emotion_temp = get_feature_keywords(taste_good_emotion, review) # 맛에 대한 긍정 감정 키워드 추출
        taste_bad_emotion_temp = get_feature_keywords(taste_bad_emotion, review) # 맛에 대한 부정 감정 키워드 추출
        good_taste2, bad_taste2 = get_taste_emotion(taste_good_emotion_temp,taste_bad_emotion_temp)
        good_taste.extend(good_taste2) # 긍정 맛 문장에 추가
        bad_taste.extend(bad_taste2) # 부정 맛 문장에 추가
        
        # 각 특성 긍부정 리스트의 크기에 따라 점수 카운트(부정은 출력 안하긴함)
        if len(good_service) > len(bad_service):
            restaurant_good_service_count += 1
        elif len(good_service) < len(bad_service):
            restaurant_bad_service_count += 1
        else:
            pass
        
        if len(good_atmosphere) > len(bad_atmosphere):
            restaurant_good_atmosphere_count += 1
        elif len(good_atmosphere) < len(bad_atmosphere):
            restaurant_bad_atmosphere_count += 1
        else:
            pass
        
        if len(good_cost) > len(bad_cost):
            restaurant_good_cost_count += 1
        elif len(good_cost) < len(bad_cost):
            restaurant_bad_cost_count += 1
        else:
            pass
            
        if len(good_visit) > len(bad_visit):
            restaurant_good_visit_count += 1
        elif len(good_visit) < len(bad_visit):
            restaurant_bad_visit_count += 1
        else:
            pass
        
        if len(good_taste) > len(bad_taste):
            restaurant_good_taste_count += 1
        elif len(good_taste) < len(bad_taste):
            restaurant_bad_taste_count += 1
        else:
            pass
        
    TT = restaurant_good_service_count + restaurant_bad_service_count + restaurant_good_taste_count + restaurant_bad_taste_count + restaurant_good_atmosphere_count + restaurant_bad_atmosphere_count + restaurant_good_cost_count + restaurant_bad_cost_count
    
    #if TT > 5: # 총 감성 카운트가 5 이상만 출력하는건데 일단 주석 처리 해 놓고 전체 출력
    print('Total review count: {}'.format(len(restaurant_review[i])))
    print('Good service: {}/{} = {}'.format(restaurant_good_service_count,restaurant_good_service_count + restaurant_bad_service_count,100*check_division(restaurant_good_service_count, restaurant_good_service_count + restaurant_bad_service_count)))
    print('Good atmosphere: {}/{} = {}'.format(restaurant_good_atmosphere_count,restaurant_good_atmosphere_count + restaurant_bad_atmosphere_count,100*check_division(restaurant_good_atmosphere_count,restaurant_good_atmosphere_count + restaurant_bad_atmosphere_count))) 
    print('Good cost: {}/{} = {}'.format(restaurant_good_cost_count,restaurant_good_cost_count + restaurant_bad_cost_count, 100*check_division(restaurant_good_cost_count,restaurant_good_cost_count + restaurant_bad_cost_count)))
    print('Good taste: {}/{} = {}'.format(restaurant_good_taste_count,restaurant_good_taste_count + restaurant_bad_taste_count,100*check_division(restaurant_good_taste_count,restaurant_good_taste_count + restaurant_bad_taste_count)))
    print('Good revisit: {}/{} = {}'.format(restaurant_good_visit_count,restaurant_good_visit_count+restaurant_bad_visit_count,100*check_division(restaurant_good_visit_count,restaurant_good_visit_count+restaurant_bad_visit_count)))
    print('')

배곧 또잇카페 배곧점
Total review count: 81
Good service: 1/2 = 50.0
Good atmosphere: 12/12 = 100.0
Good cost: 0/0 = 0
Good taste: 31/31 = 100.0
Good revisit: 0/0 = 0

배곧 카츠오모이
Total review count: 116
Good service: 4/5 = 80.0
Good atmosphere: 1/1 = 100.0
Good cost: 0/0 = 0
Good taste: 49/49 = 100.0
Good revisit: 1/1 = 100.0

배곧 양푼애등갈비
Total review count: 263
Good service: 4/5 = 80.0
Good atmosphere: 2/2 = 100.0
Good cost: 3/6 = 50.0
Good taste: 152/156 = 97.0
Good revisit: 1/1 = 100.0

배곧 투파인드피터 배곧점
Total review count: 906
Good service: 22/28 = 79.0
Good atmosphere: 111/112 = 99.0
Good cost: 40/55 = 73.0
Good taste: 471/489 = 96.0
Good revisit: 9/10 = 90.0

배곧 소쇄원 배곧본점
Total review count: 195
Good service: 5/5 = 100.0
Good atmosphere: 4/4 = 100.0
Good cost: 18/22 = 82.0
Good taste: 97/101 = 96.0
Good revisit: 2/2 = 100.0

배곧 정동진
Total review count: 1845
Good service: 78/94 = 83.0
Good atmosphere: 8/10 = 80.0
Good cost: 98/123 = 80.0
Good taste: 973/1027 = 95.0
Good revisit: 45/52 = 87.0

배곧 Tai